In [1]:
!rm -rf /data/FirstLevelAnalysis_single_run_test_2

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import sys

sys.path.append('/opt/app')

from fastfmri_toolbox.modelling.design_matrix import DesignMatrix
from fastfmri_toolbox.modelling.first_level_analysis import FirstLevelAnalysis

In [ ]:
time_window = (40,219) # Window between these timepoints in seconds
search_frequencies = [.1]
bold_path = '/data/1_attention.7T.oscprep.sub-000/ses-OPCoilCBLowerFrequencyPilot01/func/sub-000_ses-OPCoilCBLowerFrequencyPilot01_task-localizerQ1_acq-mb4_dir-RL_run-01_space-T1w_desc-preproc_bold.nii.gz'
mask_path = '/data/1_attention.7T.oscprep.sub-000/ses-OPCoilCBLowerFrequencyPilot01/func/sub-000_ses-OPCoilCBLowerFrequencyPilot01_task-localizerQ1_acq-mb4_dir-RL_run-01_space-T1w_desc-boldref_brainmask.nii.gz'
dm = DesignMatrix(time_window, search_frequencies, bold_path=bold_path)
design_matrix = dm.build_design_matrix()

fla = FirstLevelAnalysis(
    derivatives_dir = '/data/FirstLevelAnalysis_single_run_test_2',
    bold_path = bold_path,
    mask_path = mask_path,
    design_matrix = design_matrix,
    time_window = time_window,
    search_frequencies = search_frequencies
)
fla.run_frequency_glm(save_windowed_bold = True, save_predicted = True, save_residual = True)

/usr/local/lib/python3.10/dist-packages/nilearn/_utils/glm.py:335: RuntimeWarning: divide by zero encountered in divide
  return np.where(X <= 0, 0, 1. / X)


In [ ]:
!tree /data/FirstLevelAnalysis_single_run_test_2

#### Plot phase shift

In [ ]:
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import pandas as pd

TR = .3
row_idx = 0
frequency_of_interest = search_frequencies[0]

# GLM directory
glm_dir = '/data/FirstLevelAnalysis_single_run_test_2/sub-000/ses-OPCoilCBLowerFrequencyPilot01/task-localizerQ1/run-01/GLM'

# get Z-score data of frequency fit
z_score = nib.load(f"{glm_dir}/frequency-{frequency_of_interest}_z_score.nii.gz").get_fdata()
# phi 
phase_angle = nib.load(f"{glm_dir}/frequency-0.1_phaseshift.nii.gz").get_fdata()

# Coordinates of all voxels with 3> Z-score
Z_thr = 3.
coords = np.where(z_score > Z_thr)
fitted_voxels_df = pd.DataFrame(
    zip(
        z_score[coords],
        phase_angle[coords],
        *(coords)
    ),
    columns=[
        'z_score',
        'phase_angle',
        'x','y','z'
    ]
).sort_values(by='z_score',ascending=False).reset_index(drop=True)

# Convert all phase angles to be negative
# We are only interested in phase delays (negative values will shift the sine wave to the right)
convert_to_positive_phase_angle = lambda x: x-(np.pi*2) if x > 0 else x
fitted_voxels_df['phase_angle'] = fitted_voxels_df['phase_angle'].apply(convert_to_positive_phase_angle)

for dim in ['x','y','z']:
    fitted_voxels_df[dim] = fitted_voxels_df[dim].astype(int)
    
fitted_voxels_df

In [ ]:
t = dm._get_time_points(TR, time_window) # timepoints

# BOLD response
windowed = nib.load(f"{glm_dir}/windowed_bold.nii.gz").get_fdata()[
    fitted_voxels_df['x'][row_idx],
    fitted_voxels_df['y'][row_idx],
    fitted_voxels_df['z'][row_idx],
    :
]

# Predicted BOLD response
predicted = nib.load(f"{glm_dir}/predicted_bold.nii.gz").get_fdata()[
    fitted_voxels_df['x'][row_idx],
    fitted_voxels_df['y'][row_idx],
    fitted_voxels_df['z'][row_idx],
    :
]

'''
`stim_shift: phase shift to ensure the stimulus intensity begins at a minimum.
`phase_offset`: phase shift required to align stimulus wave to the predicted bold wave.
'''
A_predicted = ( predicted.max() - predicted.min() ) / 2
A_offset = ( predicted.max() + predicted.min() ) / 2
phase_offset = fitted_voxels_df['phase_angle'][row_idx]
phase_offset_s = phase_offset / (2 * np.pi * frequency_of_interest)
stimulus_shift = -np.pi/2

# Neural activity
stim_wave = A_predicted* np.sin((2 * np.pi * frequency_of_interest * t) + stimulus_shift) + A_offset

# Phase-shifted neural activity
bold_wave = A_predicted * np.sin((2 * np.pi * frequency_of_interest * t) + stimulus_shift + phase_offset) + A_offset 

In [ ]:
# Figure
fig, ax = plt.subplots(dpi = 200, figsize=(6,2))

# Plot lines
l1, = ax.plot(t,windowed,c='cyan',zorder=1,alpha=.4,lw=.75,
             label='BOLD response')
l2, = ax.plot(t,predicted,c='red',zorder=2,lw=2.,
             label='Predicted BOLD response (Modelled)')
l3, = ax.plot(t,stim_wave,c='k',zorder=3,linestyle='--',lw=1.,alpha=.4,
             label='Neural activity')
l4, = ax.plot(t,bold_wave,c='k',zorder=4,linestyle='--',lw=1.,
             label='Phase-shifted neural activity')

# Legend
ax.legend(
    handles=[l1,l2,l3,l4],
    loc='upper right',
    bbox_to_anchor=(1.,1.),
    fontsize=5,
    frameon=True,
    framealpha=1.,
    edgecolor='k',
    fancybox=False,
)

# Phase delay arrow
head_length, head_width = 1., 10.
ax.arrow(
    0,
    stim_wave.min() - stim_wave.min()*.005,
    (-1*phase_offset_s)-head_length,
    0,
    head_width=10.,
    head_length=head_length,fc='blue',ec='blue',lw=1.
)
ax.text(
    0,
    stim_wave.min() - stim_wave.min()*.012,
    f'{(-1 * phase_offset_s):.2f} s',
    fontsize=5,c='blue'
)